In [1]:
import pandas as pd
import numpy as np
import os
from openai import OpenAI
from dotenv import load_dotenv

In [23]:
load_dotenv()
%matplotlib inline
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [2]:
df = pd.read_csv('features_full_final.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111956 entries, 0 to 111955
Data columns (total 82 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   mbid                                 111956 non-null  object 
 1   title                                111956 non-null  object 
 2   artist                               111952 non-null  object 
 3   genre                                111956 non-null  object 
 4   year                                 111956 non-null  int64  
 5   duration_ms                          111576 non-null  float64
 6   high_danceability_value              111753 non-null  object 
 7   high_danceability_probability        111753 non-null  float64
 8   high_gender_value                    111753 non-null  object 
 9   high_gender_probability              111753 non-null  float64
 10  high_genre_dortmund_value            111753 non-null  object 
 11  high_genre_do

In [26]:
df.columns

Index(['mbid', 'title', 'artist', 'genre', 'year', 'duration_ms',
       'high_danceability_value', 'high_danceability_probability',
       'high_gender_value', 'high_gender_probability',
       'high_genre_dortmund_value', 'high_genre_dortmund_probability',
       'high_genre_electronic_value', 'high_genre_electronic_probability',
       'high_genre_rosamerica_value', 'high_genre_rosamerica_probability',
       'high_genre_tzanetakis_value', 'high_genre_tzanetakis_probability',
       'high_ismir04_rhythm_value', 'high_ismir04_rhythm_probability',
       'high_mood_acoustic_value', 'high_mood_acoustic_probability',
       'high_mood_aggressive_value', 'high_mood_aggressive_probability',
       'high_mood_electronic_value', 'high_mood_electronic_probability',
       'high_mood_happy_value', 'high_mood_happy_probability',
       'high_mood_party_value', 'high_mood_party_probability',
       'high_mood_relaxed_value', 'high_mood_relaxed_probability',
       'high_mood_sad_value', 'high_m

In [27]:
def filtrar_features(df:pd.DataFrame):
    ritmo_energia = [
        'low_bpm', 'low_onset_rate', 'low_danceability',
        'low_dynamic_complexity', 'low_average_loudness'
    ]
    tonalidad = [
        'low_key_strength', 'low_chords_changes_rate', 'low_chords_number_rate'
    ]
    mfccs = [f'low_mfcc_mean_{i}' for i in range(13)]
    moods = [
        'high_mood_acoustic_probability',
        'high_mood_aggressive_probability',
        'high_mood_electronic_probability',
        'high_mood_happy_probability',
        'high_mood_party_probability',
        'high_mood_relaxed_probability',
        'high_mood_sad_probability'
    ]
    columnas = ['genre'] + ritmo_energia + tonalidad + mfccs + moods
    return df[columnas].copy()

In [28]:
datos = filtrar_features(df)

In [29]:
datos.shape

(111956, 29)

In [30]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111956 entries, 0 to 111955
Data columns (total 29 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   genre                             111956 non-null  object 
 1   low_bpm                           111955 non-null  float64
 2   low_onset_rate                    111955 non-null  float64
 3   low_danceability                  111955 non-null  float64
 4   low_dynamic_complexity            111956 non-null  float64
 5   low_average_loudness              111956 non-null  float64
 6   low_key_strength                  111899 non-null  float64
 7   low_chords_changes_rate           111955 non-null  float64
 8   low_chords_number_rate            111955 non-null  float64
 9   low_mfcc_mean_0                   111956 non-null  float64
 10  low_mfcc_mean_1                   111956 non-null  float64
 11  low_mfcc_mean_2                   111956 non-null  f

In [31]:
datos.dropna(inplace=True)
datos.shape

(111752, 29)

In [32]:
def promedio_genero(df_filtrado, genero):
    return df_filtrado[df_filtrado['genre'] == genero].mean(numeric_only=True)

In [33]:
promedio_genero(datos, 'rock')

low_bpm                             126.046423
low_onset_rate                        3.222548
low_danceability                      1.117270
low_dynamic_complexity                3.838504
low_average_loudness                  0.799274
low_key_strength                      0.706085
low_chords_changes_rate               0.062181
low_chords_number_rate                0.002559
low_mfcc_mean_0                    -669.521063
low_mfcc_mean_1                     131.665669
low_mfcc_mean_2                      -2.986507
low_mfcc_mean_3                      25.324911
low_mfcc_mean_4                       4.910339
low_mfcc_mean_5                       5.981352
low_mfcc_mean_6                       0.229045
low_mfcc_mean_7                       3.361125
low_mfcc_mean_8                      -0.777716
low_mfcc_mean_9                       1.304612
low_mfcc_mean_10                     -1.822891
low_mfcc_mean_11                     -0.659284
low_mfcc_mean_12                     -1.810811
high_mood_aco

In [34]:
def comparar_cancion_con_promedio(nueva_cancion:pd.Series, promedio_genero:pd.Series) -> dict:
    nueva_cancion.drop(['genre'], inplace=True)
    diferencias = {}
    for feature in nueva_cancion.index:
        valor_actual = nueva_cancion[feature]
        valor_referencia = promedio_genero[feature]
        diferencias[feature] = round(float(valor_actual-valor_referencia),3)
    return diferencias

In [35]:
nueva_cancion = filtrar_features(df.iloc[123])

In [36]:
genero = nueva_cancion['genre']
genero

'rock'

In [37]:
promedios = promedio_genero(datos, nueva_cancion['genre'])

In [38]:
diferencias = comparar_cancion_con_promedio(nueva_cancion,promedios)

In [39]:
diferencias

{'low_bpm': -14.189,
 'low_onset_rate': -0.152,
 'low_danceability': 0.064,
 'low_dynamic_complexity': 0.085,
 'low_average_loudness': 0.124,
 'low_key_strength': 0.042,
 'low_chords_changes_rate': 0.004,
 'low_chords_number_rate': -0.0,
 'low_mfcc_mean_0': 1.02,
 'low_mfcc_mean_1': -24.764,
 'low_mfcc_mean_2': 23.814,
 'low_mfcc_mean_3': 7.773,
 'low_mfcc_mean_4': 8.52,
 'low_mfcc_mean_5': 1.696,
 'low_mfcc_mean_6': 3.59,
 'low_mfcc_mean_7': -3.046,
 'low_mfcc_mean_8': 0.01,
 'low_mfcc_mean_9': 0.331,
 'low_mfcc_mean_10': -5.199,
 'low_mfcc_mean_11': -0.682,
 'low_mfcc_mean_12': -3.176,
 'high_mood_acoustic_probability': 0.002,
 'high_mood_aggressive_probability': 0.108,
 'high_mood_electronic_probability': -0.177,
 'high_mood_happy_probability': -0.205,
 'high_mood_party_probability': -0.031,
 'high_mood_relaxed_probability': -0.252,
 'high_mood_sad_probability': -0.11}

In [40]:
def generar_prompt(genero, diferencias, nueva_cancion, promedio_genero, porcentaje_umbral=0.05):
    sugerencias = []

    for feature, diferencia in diferencias.items():
        val_actual = nueva_cancion[feature]
        val_prom = promedio_genero[feature]

        umbral = abs(val_prom * porcentaje_umbral) if val_prom != 0 else porcentaje_umbral

        if abs(diferencia) > umbral:
            if diferencia < 0:
                sugerencias.append(
                    f"- '{feature}': el valor actual es {val_actual:.2f}, mientras que el promedio del género es {val_prom:.2f}. Está por debajo del promedio; podrías intentar incrementarlo.")
            else:
                sugerencias.append(
                    f"- '{feature}': el valor actual es {val_actual:.2f}, mientras que el promedio del género es {val_prom:.2f}. Está por encima del promedio; podrías ajustarlo.")

    if not sugerencias:
        sugerencias.append("Los valores están bastante alineados con el promedio del género.")

    prompt = f"Tengo una canción del género {genero}. Estas son las diferencias respecto al promedio de las canciones más populares del mismo género:\n\n"
    prompt += "\n".join(sugerencias)
    prompt += "\n\nDame recomendaciones musicales para mejorar esta canción"
    prompt += "\nLos nombres de las features vienen en inglés, pero la respuesta debe ser todo en castellano."

    return prompt

In [41]:
promt = generar_prompt(genero, diferencias, nueva_cancion, promedios)

In [42]:
print(promt)

Tengo una canción del género rock. Estas son las diferencias respecto al promedio de las canciones más populares del mismo género:

- 'low_bpm': el valor actual es 111.86, mientras que el promedio del género es 126.05. Está por debajo del promedio; podrías intentar incrementarlo.
- 'low_danceability': el valor actual es 1.18, mientras que el promedio del género es 1.12. Está por encima del promedio; podrías ajustarlo.
- 'low_average_loudness': el valor actual es 0.92, mientras que el promedio del género es 0.80. Está por encima del promedio; podrías ajustarlo.
- 'low_key_strength': el valor actual es 0.75, mientras que el promedio del género es 0.71. Está por encima del promedio; podrías ajustarlo.
- 'low_chords_changes_rate': el valor actual es 0.07, mientras que el promedio del género es 0.06. Está por encima del promedio; podrías ajustarlo.
- 'low_mfcc_mean_1': el valor actual es 106.90, mientras que el promedio del género es 131.67. Está por debajo del promedio; podrías intentar in

In [43]:
def obtener_recomendacion(prompt):
    response = client.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[
            {"role": "system", "content": "Eres una app de música que brinda consejos claros y útiles sobre cómo mejorar una canción según sus características."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=1000
    )
    return response.choices[0].message.content

In [44]:
import time

inicio = time.time()
recomendacion = obtener_recomendacion(promt)
fin = time.time()

print(f"Tiempo de ejecución: {fin - inicio:.2f} segundos")

Tiempo de ejecución: 7.91 segundos


In [45]:
print(recomendacion)

Para mejorar tu canción de rock y acercarla más a las características de las canciones más populares del género, te recomiendo lo siguiente:

1. Aumenta el tempo (BPM): Actualmente está en 111.86, mientras que el promedio es 126.05. Incrementar el tempo le dará más energía y dinamismo.

2. Ajusta la capacidad de baile (danceability): Está un poco por encima del promedio. Podrías buscar un balance para que la canción mantenga un buen groove sin perder la esencia rockera.

3. Modera la sonoridad promedio (loudness): Está ligeramente más alta que el promedio. Reducir un poco el volumen general puede ayudar a evitar saturación y mejorar la claridad.

4. Ajusta la fuerza de la tonalidad (key strength): Al estar por encima del promedio, podrías suavizar la estabilidad tonal para que sea más atractiva y menos predecible.

5. Reduce un poco la tasa de cambios de acordes (chords changes rate): Está un poco más alta que la media. Considera simplificar algunos cambios para mejorar la coherencia m

In [46]:
df['high_danceability_probability']

0         0.741577
1         0.549704
2         0.862171
3         0.595985
4         0.903194
            ...   
111951    0.773452
111952    0.904865
111953    0.965314
111954    0.776017
111955    0.769704
Name: high_danceability_probability, Length: 111956, dtype: float64

In [ ]:
from api_reporte.recomendaciones import filtrar_features

In [4]:
df.shape

(111956, 82)

In [5]:
df = filtrar_features(df)
df.dropna(inplace=True)
df.shape

(111752, 28)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 111752 entries, 0 to 111955
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   genre                             111752 non-null  object 
 1   low_bpm                           111752 non-null  float64
 2   low_onset_rate                    111752 non-null  float64
 3   low_danceability                  111752 non-null  float64
 4   low_dynamic_complexity            111752 non-null  float64
 5   low_average_loudness              111752 non-null  float64
 6   high_danceability_probability     111752 non-null  float64
 7   low_key_strength                  111752 non-null  float64
 8   low_chords_changes_rate           111752 non-null  float64
 9   low_mfcc_mean_0                   111752 non-null  float64
 10  low_mfcc_mean_1                   111752 non-null  float64
 11  low_mfcc_mean_2                   111752 non-null  float6

In [51]:
df = df.groupby('genre').mean(numeric_only=True).reset_index()

In [52]:
df.to_csv('promedio_features.csv', index=False)

In [11]:
resultado.json()

{'recomendacion': {'mbid': 'f15c6cb7-6000-4b98-99dc-6a69987371e3',
  'title': 'CMOS Therapy',
  'artist': 'Deepchord',
  'genre': 'electronic',
  'year': 2015,
  'duration_ms': 436000.0,
  'high_danceability_value': 'danceable',
  'high_danceability_probability': 0.999988913536,
  'high_gender_value': 'female',
  'high_gender_probability': 0.572697758675,
  'high_genre_dortmund_value': 'electronic',
  'high_genre_dortmund_probability': 0.999999344349,
  'high_genre_electronic_value': 'techno',
  'high_genre_electronic_probability': 0.723772108555,
  'high_genre_rosamerica_value': 'rhy',
  'high_genre_rosamerica_probability': 0.562672436237,
  'high_genre_tzanetakis_value': 'jaz',
  'high_genre_tzanetakis_probability': 0.315071135759,
  'high_ismir04_rhythm_value': 'VienneseWaltz',
  'high_ismir04_rhythm_probability': 0.496150821447,
  'high_mood_acoustic_value': 'not_acoustic',
  'high_mood_acoustic_probability': 0.999982893467,
  'high_mood_aggressive_value': 'aggressive',
  'high_moo

In [ ]:
from api_reporte.recomendaciones import *
datos = pd.read_csv('promedio_features.csv')
datos.shape

(20, 28)

In [ ]:
from api_reporte.api_recomendaciones import Cancion

def brindar_recomendacion(input:Cancion):
    datos = pd.read_csv('promedio_features.csv')
    nueva_cancion = filtrar_features(pd.Series(input))
    genero = obtener_genero(nueva_cancion)
    promedio_genero_nueva_cancion = promedio_genero(datos, genero)
    diferencias = comparar_cancion_con_promedio(nueva_cancion,promedio_genero_nueva_cancion)
    prompt = generar_prompt(genero, diferencias, nueva_cancion, promedio_genero_nueva_cancion)
    return prompt

In [4]:
datos = filtrar_features(datos)
datos.shape

(20, 28)

In [5]:
df = pd.read_csv('features_full_final.csv').dropna()
prueba = df.sample(n=1).squeeze().to_dict()
len(prueba.keys())

82

In [11]:
print(brindar_recomendacion(prueba))

Tengo una canción del género folk. Estas son las diferencias respecto al promedio de las canciones más populares del mismo género:

- 'low_onset_rate': el valor actual es 4.21, mientras que el promedio del género es 3.26. Está por encima del promedio; podrías ajustarlo.
- 'low_danceability': el valor actual es 1.23, mientras que el promedio del género es 1.07. Está por encima del promedio; podrías ajustarlo.
- 'low_dynamic_complexity': el valor actual es 3.30, mientras que el promedio del género es 4.73. Está por debajo del promedio; podrías intentar incrementarlo.
- 'low_average_loudness': el valor actual es 0.93, mientras que el promedio del género es 0.68. Está por encima del promedio; podrías ajustarlo.
- 'high_danceability_probability': el valor actual es 0.98, mientras que el promedio del género es 0.87. Está por encima del promedio; podrías ajustarlo.
- 'low_key_strength': el valor actual es 0.82, mientras que el promedio del género es 0.75. Está por encima del promedio; podrías

In [6]:
prueba

{'mbid': '1da3beaa-c26f-4c63-a558-1790c407a984',
 'title': 'Between Waves',
 'artist': 'The Album Leaf',
 'genre': 'electronic',
 'year': 2016,
 'duration_ms': 368694.0,
 'high_danceability_value': 'not_danceable',
 'high_danceability_probability': 0.862999856472,
 'high_gender_value': 'male',
 'high_gender_probability': 0.510834634304,
 'high_genre_dortmund_value': 'electronic',
 'high_genre_dortmund_probability': 0.995353221893,
 'high_genre_electronic_value': 'ambient',
 'high_genre_electronic_probability': 0.570509672165,
 'high_genre_rosamerica_value': 'pop',
 'high_genre_rosamerica_probability': 0.660472452641,
 'high_genre_tzanetakis_value': 'jaz',
 'high_genre_tzanetakis_probability': 0.264038801193,
 'high_ismir04_rhythm_value': 'VienneseWaltz',
 'high_ismir04_rhythm_probability': 0.279851645231,
 'high_mood_acoustic_value': 'acoustic',
 'high_mood_acoustic_probability': 0.560010373592,
 'high_mood_aggressive_value': 'not_aggressive',
 'high_mood_aggressive_probability': 0.992

In [3]:
entrada = {
"genre": "rock",
"duration_ms": 253974,
"high_danceability_value": "not_danceable",
"high_danceability_probability": 0.5838052034378052,
"high_gender_value": "male",
"high_gender_probability": 0.5807787775993347,
"high_mood_acoustic_value": "non_acoustic",
"high_mood_acoustic_probability": 0.889371395111084,
"high_mood_aggressive_value": "aggressive",
"high_mood_aggressive_probability": 0.6102733612060547,
"high_mood_electronic_value": "electronic",
"high_mood_electronic_probability": 0.7434790134429932,
"high_mood_happy_value": "non_happy",
"high_mood_happy_probability": 0.9543348550796509,
"high_mood_party_value": "non_party",
"high_mood_party_probability": 0.8928609490394592,
"high_mood_relaxed_value": "relaxed",
"high_mood_relaxed_probability": 0.6134181022644043,
"high_mood_sad_value": "non_sad",
"high_mood_sad_probability": 0.7707998752593994,
"high_moods_mirex_value": "Cluster5",
"high_moods_mirex_probability": 0.6387696266174316,
"high_timbre_value": "bright",
"high_timbre_probability": 0.5203741788864136,
"high_tonal_atonal_value": "atonal",
"high_tonal_atonal_probability": 0.9881431460380554,
"high_voice_instrumental_value": "voice",
"high_voice_instrumental_probability": 0.5408473014831543,
"low_average_loudness": 18.720955833591585,
"low_dynamic_complexity": 0.0,
"low_mfcc_mean_0": -605.623046875,
"low_mfcc_mean_1": 148.2161865234375,
"low_mfcc_mean_2": 1.0899323225021362,
"low_mfcc_mean_3": 33.8800163269043,
"low_mfcc_mean_4": 9.242630958557129,
"low_mfcc_mean_5": 12.825408935546875,
"low_mfcc_mean_6": -10.752599716186523,
"low_mfcc_mean_7": 8.433829307556152,
"low_mfcc_mean_8": -2.8856284618377686,
"low_mfcc_mean_9": 1.9817363023757935,
"low_mfcc_mean_10": 2.6789426803588867,
"low_mfcc_mean_11": -0.10977625846862793,
"low_mfcc_mean_12": -1.8002312183380127,
"audio_sample_rate": 44100.0,
"audio_codec": "pcm_s24le",
"audio_bit_rate": 2116800,
"audio_equal_loudness": 0.0,
"audio_analysis_sample_rate": 44100.0,
"audio_length": 253.97469387755103,
"audio_md5_encoded": "77792e8ef05b8a7a51989b4b494c2c2b",
"audio_replay_gain": -0.00042015998042188585,
"audio_downmix": "mix",
"audio_lossless": True,
"low_key_key": "E",
"low_key_scale": "major",
"low_key_strength": 0.9335505962371826,
"low_chords_scale": "major",
"low_chords_changes_rate": 0.06544789671897888,
"low_chords_key": "A",
"low_tuning_frequency": 440.76312255859375,
"low_danceability": 1.37296724319458,
"low_onset_rate": 1.8227664232254028,
"low_bpm": 129.63111877441406,
"low_beats_count": 543
}

In [4]:
len(entrada.keys())

64

In [8]:
import requests

url = 'https://recomendador-api-173219828681.us-central1.run.app/recomendar'
headers = {"Content-Type": "application/json"}

In [9]:
%%time

resultado = requests.post(url, json=entrada, headers=headers)
resultado.status_code

CPU times: user 11.3 ms, sys: 6.01 ms, total: 17.3 ms
Wall time: 1min 1s


200

In [9]:
print(resultado.json())

{'recomendacion': 'Para mejorar tu canción electrónica y acercarla más a las características de las canciones más populares del género, te recomiendo lo siguiente:\n\n1. **Incrementar el BPM (tempo):** Tu canción tiene un BPM de 94.04, que es significativamente más bajo que el promedio de 122.93. Subir el tempo la hará más energética y adecuada para el género electronic, lo que puede aumentar su atractivo.\n\n2. **Ajustar la tasa de onset:** Actualmente está un poco por encima del promedio. Puedes experimentar con reducir ligeramente la cantidad de ataques o eventos rítmicos para que la canción tenga una sensación más equilibrada y no sobrecargada.\n\n3. **Mejorar la danceabilidad:** Tanto la danceabilidad como la probabilidad de danceabilidad están por debajo del promedio. Para ello, utiliza ritmos más marcados y grooves que inviten a moverse, además de simplificar o mejorar la coherencia rítmica para facilitar el baile.\n\n4. **Ajustar la complejidad dinámica y la sonoridad:** La din

In [10]:
print(resultado.json()['recomendacion'])

Para mejorar tu canción de rock y acercarla más al perfil de las canciones más populares del género, te recomiendo lo siguiente:

1. Incrementa la tasa de ataques (low_onset_rate): Esto puede lograrse haciendo que los instrumentos, especialmente la batería y la guitarra, tengan más ataques claros y definidos. Considera agregar golpes más marcados o riffs con mayor percusión para aumentar la energía.

2. Ajusta la bailabilidad (low_danceability): Actualmente es un poco más alta que el promedio, por lo que podrías simplificar o moderar el ritmo para que sea ligeramente menos bailable, manteniendo el carácter rockero.

3. Incrementa la complejidad dinámica (low_dynamic_complexity): Añade variaciones en volumen, intensidad y expresión a lo largo de la canción para hacerla más interesante. Juega con cambios de dinámica en secciones o con efectos que varíen la textura sonora.

4. Ajusta el volumen promedio (low_average_loudness): Tu canción es más alta que el promedio, por lo que bajarla un 

In [5]:
for clave, valor in entrada.items():
    print(f"{clave}: {type(valor).__name__}")

genre: str
duration_ms: int
high_danceability_value: str
high_danceability_probability: float
high_gender_value: str
high_gender_probability: float
high_mood_acoustic_value: str
high_mood_acoustic_probability: float
high_mood_aggressive_value: str
high_mood_aggressive_probability: float
high_mood_electronic_value: str
high_mood_electronic_probability: float
high_mood_happy_value: str
high_mood_happy_probability: float
high_mood_party_value: str
high_mood_party_probability: float
high_mood_relaxed_value: str
high_mood_relaxed_probability: float
high_mood_sad_value: str
high_mood_sad_probability: float
high_moods_mirex_value: str
high_moods_mirex_probability: float
high_timbre_value: str
high_timbre_probability: float
high_tonal_atonal_value: str
high_tonal_atonal_probability: float
high_voice_instrumental_value: str
high_voice_instrumental_probability: float
low_average_loudness: float
low_dynamic_complexity: float
low_mfcc_mean_0: float
low_mfcc_mean_1: float
low_mfcc_mean_2: float
low

In [6]:
ritmo_energia = [
    'low_bpm', 'low_onset_rate', 'low_danceability',
    'low_dynamic_complexity', 'low_average_loudness', 'high_danceability_probability'
]
tonalidad = [
    'low_key_strength', 'low_chords_changes_rate'
]
mfccs = [f'low_mfcc_mean_{i}' for i in range(13)]
moods = [
    'high_mood_acoustic_probability',
    'high_mood_aggressive_probability',
    'high_mood_happy_probability',
    'high_mood_party_probability',
    'high_mood_relaxed_probability',
    'high_mood_sad_probability'
]
columnas = ['genre'] + ritmo_energia + tonalidad + mfccs + moods

In [7]:
for columna in columnas:
    if columna in entrada.keys():
        print(f"✅ {columna}")
    else:
        print(f"❌ {columna}")

✅ genre
✅ low_bpm
✅ low_onset_rate
✅ low_danceability
✅ low_dynamic_complexity
✅ low_average_loudness
✅ high_danceability_probability
✅ low_key_strength
✅ low_chords_changes_rate
✅ low_mfcc_mean_0
✅ low_mfcc_mean_1
✅ low_mfcc_mean_2
✅ low_mfcc_mean_3
✅ low_mfcc_mean_4
✅ low_mfcc_mean_5
✅ low_mfcc_mean_6
✅ low_mfcc_mean_7
✅ low_mfcc_mean_8
✅ low_mfcc_mean_9
✅ low_mfcc_mean_10
✅ low_mfcc_mean_11
✅ low_mfcc_mean_12
✅ high_mood_acoustic_probability
✅ high_mood_aggressive_probability
✅ high_mood_happy_probability
✅ high_mood_party_probability
✅ high_mood_relaxed_probability
✅ high_mood_sad_probability
